### В данном блокноте рассматривается применение генетического алгоритма в четкой постановке задачи о назначениях

Подключаем необходимые библиотеки

In [ ]:
import numpy as np
import random
from tabulate import tabulate

Вычисляем фитнес-функцию, основанную на целевой функции из постановки задачи. \
\begin{equation}
f_k = \sum_{j \in J} c_{s_{kj}j}
\end{equation}
Суммируем каждый ген выбранной хромосомы (возможного решения).

In [ ]:
def fitness(chromosome, c_matrix):
    return sum(c_matrix[i][j] for j, i in enumerate(chromosome))

Вычисляем функцию непригодности. \
\begin{equation}
u_k = \sum_{i \in I} \max\left[ 0, \left( \sum_{j \in J, s_{kj} = i} r_{ij} \right) - b_i \right]
\end{equation}
Если $ u_k = 0 $, то считаем, что рассматриваемая хромосома (решение) удовлетворяет введенным ограничениям.

В дальнейшем хромосома с наибольшим значениям функции непригодности заменяется созданным дочерним решением.

In [ ]:
def unfitness(chromosome, r_matrix, b_list):
    unfitness_value = 0
    agent_resources = np.zeros(len(b_list))

    for j, i in enumerate(chromosome):
        agent_resources[i] += r_matrix[i][j]

    for i in range(len(b_list)):
        if agent_resources[i] > b_list[i]:
            unfitness_value += agent_resources[i] - b_list[i]

    return unfitness_value

Создаем популяцию. Количество особей N задает пользователем.

In [ ]:
def generate_population(N, num_agents, num_tasks):
    return [np.random.randint(0, num_agents, num_tasks) for _ in range(N)]

Для отбора двух родительских хромосом используем метод бинарного турнира.

In [ ]:
import random

def tournament_selection(population, c_matrix, r_matrix, b_list):
    def select_one(population, c_matrix, r_matrix, b_list):
        selected = random.sample(population, 2)
        fit_1 = fitness(selected[0], c_matrix)
        fit_2 = fitness(selected[1], c_matrix)

        if unfitness(selected[0], r_matrix, b_list) == 0 and unfitness(selected[1], r_matrix, b_list) == 0:
            return selected[0] if fit_1 > fit_2 else selected[1]

        if unfitness(selected[0], r_matrix, b_list) == 0:
            return selected[0]

        if unfitness(selected[1], r_matrix, b_list) == 0:
            return selected[1]

        return selected[0] if fit_1 > fit_2 else selected[1]

    parent1 = select_one(population, c_matrix, r_matrix, b_list)

    while True:
        parent2 = select_one(population, c_matrix, r_matrix, b_list)
        if not np.array_equal(parent2, parent1):
            break

    return parent1, parent2

Создаем дочернюю хромосому, используя гены отобранных родителей.

In [ ]:
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    if random.random() > 0.5:
        child = np.concatenate([parent1[:point], parent2[point:]])
    else:
        child = np.concatenate([parent2[:point], parent1[point:]])
    return child

Применяем мутацию генов дочерней хромосомы.

In [ ]:
def mutate(chromosome, num_agents):
    i = random.randint(0, len(chromosome) - 1)
    chromosome[i] = random.randint(0, num_agents - 1)
    return chromosome

Запускаем генетический алгоритм.
Число создания хромосом без улучшения целевой функции M задается пользователем.

In [ ]:
def genetic_algorithm(num_agents, num_tasks, c_matrix, r_matrix, b_list, N, M):
    population = generate_population(N, num_agents, num_tasks)

    best_solution = None
    best_fitness = float('-inf')
    best_unfitness = float('inf')

    no_improvement_counter = 0

    while no_improvement_counter < M:
        population_fitness = [(chromosome, fitness(chromosome, c_matrix), unfitness(chromosome, r_matrix, b_list)) for chromosome in population]

        population_fitness.sort(key=lambda x: (x[2], -x[1]))

        if population_fitness[0][2] == 0 and population_fitness[0][1] > best_fitness:
            best_solution = population_fitness[0][0]
            best_fitness = population_fitness[0][1]
            best_unfitness = 0
            no_improvement_counter = 0
        else:
            no_improvement_counter += 1

        parent1, parent2 = tournament_selection(population, c_matrix, r_matrix, b_list)

        child = crossover(parent1, parent2)

        if random.random() < 0.1:
            child = mutate(child, num_agents)

        child_fitness = fitness(child, c_matrix)
        child_unfitness = unfitness(child, r_matrix, b_list)

        if child_unfitness == 0:
            if all(not np.array_equal(child, chrom) for chrom, _, _ in population_fitness):
                population_fitness[-1] = (child, child_fitness, child_unfitness)
        elif child_unfitness < population_fitness[-1][2]:
            population_fitness[-1] = (child, child_fitness, child_unfitness)

        population = [chrom for chrom, _, _ in population_fitness]

    return best_solution, best_fitness

Считывание вводного файла.

In [ ]:
import pandas as pd

def read_data_from_excel(file_path):
    # Чтение Excel-файла
    df = pd.read_excel(file_path, header=None)

    # Количество агентов (сотрудников)
    num_agents = int(df.iloc[0, 0])

    # Количество задач
    num_tasks = int(df.iloc[2, 0])

    # Матрица эффективности C (возможны нецелые числа)
    c_matrix = df.iloc[4:4+num_agents, 0:num_tasks].astype(float).values.tolist()

    # Матрица ресурсов R (возможны нецелые числа)
    r_matrix = df.iloc[4+num_agents+1:4+num_agents+1+num_agents, 0:num_tasks].astype(float).values.tolist()

    # Вектор доступных ресурсов B (возможны нецелые числа)
    b_list = df.iloc[4+num_agents+1+num_agents+1:4+num_agents+1+num_agents+num_agents+1, 0].astype(float).tolist()
    
    # Количество особей в популяции N (целое число)
    N = int(df.iloc[4+num_agents+1+num_agents+num_agents+2, 0])

    # Количество итераций без улучшений M (целое число)
    M = int(df.iloc[4+num_agents+1+num_agents+num_agents+4, 0])

    return num_agents, num_tasks, c_matrix, r_matrix, b_list, N, M

Вывод результатов. \
Для лучшей визуализации результатов выводим таблицу с соотношениями: Номер задачи - Номер назначенного сотрудника

In [ ]:
def print_solution_table(solution, num_agents, num_tasks):
    assignment_matrix = np.zeros((num_agents, num_tasks), dtype=int)

    for j, i in enumerate(solution):
        assignment_matrix[i][j] = 1

    headers = [f'Задача {j + 1}' for j in range(num_tasks)]

    print("\nРаспределение задач между сотрудниками:")
    print(tabulate(assignment_matrix, headers=headers, tablefmt="grid"))

Запуск алгоритма.

Данные вводятся в файл xlsx в следующем порядке:
1. Количество агентов (сотрудников)
2. Количество задач
3. Матрица эффективности C размером [число агентов] x [число задач]
4. Матрица ресурсов R, которые нужны для каждый задачи, размером [число агентов] x [число задач]
5. Вектор доступных ресурсов размером [число агентов]
6. Количество особей в популяции N
7. Количество итераций без улучшений M

Каждый пункт должен заполняться через строку после предыдущего и начинаться с первой ячейки в строке. Первый пункт заполяется в 1 строке и 1 столбце.


Результат выводится в эту же таблицу в формате:
1. Таблица подобранного решения [номер задачи] - [номер назначенного сотрудника]
2. Результат максимальной эффективности

In [ ]:
if __name__ == "__main__":

    file_path = r"--путь до файла--
    num_agents, num_tasks, c_matrix, r_matrix, b_list, N, M = read_data_from_excel(file_path)

    best_solution, best_fitness = genetic_algorithm(num_agents, num_tasks, c_matrix, r_matrix, b_list, N, M)

    print_solution_table(best_solution, num_agents, num_tasks)
    print(f"\nМаксимальная эффективность: {best_fitness}")